In [3]:
import sqlite3

import pandas as pd

In [4]:
pd.options.display.max_rows = 500

In [5]:
class SQLQueryExecuter():
    
    def __init__(self, filepath):
        self.filepath = filepath
    
    def __enter__(self):
        self.con = sqlite3.connect(self.filepath)
        self.cur = self.con.cursor()     
        
        return self 
    
    def __exit__(self, *exc):
        self.cur.close()
        self.con.close()
        
    def get_df_from_cursor_result(self):
        if self.cur.description:
            column_names = [x[0] for x in self.cur.description]
            records = [row for row in self.cur]
            return pd.DataFrame(records, columns=column_names)
        else:
            raise ValueError('')
    
    def get_query_results(self, query_string):
        self.cur.execute(query_string)
        return self.get_df_from_cursor_result()
    

In [6]:
class SQLTableMaker():
    
    def __init__(self, filepath, table_name, table_column_name_and_type_dict):
        self.filepath = filepath
        self.table_name = table_name
        self.table_column_name_and_type_dict = table_column_name_and_type_dict
        
    def __enter__(self):
        self.con = sqlite3.connect(self.filepath)
        self.cur = self.con.cursor()     
        
        return self 
    
    def __exit__(self, *exc):
        self.cur.close()
        self.con.close()
        
    def add_table(self):
        
        table_column_name_string = ', '.join(f'"{key}" {value}' for key, value in self.table_column_name_and_type_dict.items())
        
        self.cur.execute(f'drop table if exists {self.table_name}')
        self.cur.execute(f'create Table {self.table_name}({table_column_name_string})')
        
    def add_row_to_table_from_column_names_and_value_tuple(self, column_names, value_tuple, placeholder=None, number_of_columns=None):
        
        
        if not placeholder:
            if number_of_columns:
                placeholder = ', '.join(['?' for _ in range(number_of_columns)])
            else:
                raise ValueError('Must provide either placeholder or number_of_columns')
            
        query = f'INSERT INTO {self.table_name} ({column_names}) VALUES ({placeholder})'
        
        try: 
            self.cur.execute(query, value_tuple)
        except sqlite3.OperationalError as err:
            print(f'insert error: {err}')
            
    def add_multiple_rows_to_table_from_column_names_and_list_of_value_tuples(self, column_names, number_of_columns, list_of_value_tuples):
    
        placeholder = ', '.join(['?' for _ in range(number_of_columns)])
        query = f'INSERT INTO {self.table_name} ({column_names}) VALUES ({placeholder})'
        
        try: 
            self.cur.executemany(query, list_of_value_tuples)
        except sqlite3.OperationalError as err:
            print(f'insert error: {err}')
    
    def commit(self):
        self.con.commit()
            

In [7]:
products_df = pd.read_csv('Products1.txt', sep='|')
products_df['Size'] = products_df['Size'].str.replace('[^\x00-\x7F]', ' ', regex=True)
products_df['base_price_dollar'] = products_df['BasePrice'].str.extract('\$(.+)')
products_df['base_price_dollar'] = pd.to_numeric(products_df['base_price_dollar'])
products_df

,Manufacturer,Product Name,Size,itemType,SKU,BasePrice,base_price_dollar
0,Zatarains,Jambalaya Rice Mix,12 oz,Rice/Rice Mix,42081001,$2.49,2.49
1,Zatarains,Jambalaya Rice Mix,8 oz,Rice/Rice Mix,42082001,$1.79,1.79
2,Yucatan,Guacamole Regular,8 oz,NaN,42083001,$3.99,3.99
3,Yuban,Coffee Original Blend,12 oz,Coffee/Creamer,42084001,$3.99,3.99
4,Yoplait,GoGurt Variety Pack,8 ct,Yogurt,42085001,$2.99,2.99
...,...,...,...,...,...,...,...
2070,Aidells,Meatballs Pineapple Teriyaki,12 oz,Frozen Food,44156001,$6.59,6.59
2071,Aidells,Meatballs Sundried Tomato,12 oz,Frozen Food,44157001,$6.59,6.59
2072,Act II,Popcorn Butter,6 pkg,Frozen Food,44158001,$3.99,3.99
2073,A Taste Of Thai,Coconut Milk Unsweetened,13.5 oz,Drink,44159001,$3.59,3.59


### Combine the databases 

In [8]:
table_and_column_names_dict = {
    '1' : ['Transactions_Inv', {
        'trans_date': 'date',
        'customer_number': 'customer_number',
        'SKU': 'sku',
        'price': 'sale_price',
        'left_in_stock': 'items_left',
        'cases_bought': 'total_cases_ordered',
     }],
    '2' : ['transacStockandCases', {
        'trans_date': 'date',
        'customer_ID': 'customer_number',
        'SKU': 'sku',
        'salePrice': 'sale_price',
        'itemsLeft': 'items_left',
        'caseOrdered': 'total_cases_ordered',
    }],
    '3' : ['transactions', {
        'Date': 'date',
        'Customer #': 'customer_number',
        'SKU': 'sku',
        'Sale Price': 'sale_price',
        'Items Left': 'items_left',
        'Total Cases Ordered': 'total_cases_ordered',
    }]
}

### Just make sure column names are correct. Sort of an end-to-end test to make sure all database are readable

In [9]:
for team, (table, column_name_dict) in table_and_column_names_dict.items():
    
    database_address = f'./../hw5/databases/Team{team}_grocery.db'

    with SQLQueryExecuter(database_address) as sql_query_executer:

        query = f'''
        SELECT *
        FROM {table}
        LIMIT 10
        '''

        transactions_df = sql_query_executer.get_query_results(query)
        print(team, 'Before', list(transactions_df.columns))
        transactions_df = transactions_df.rename(column_name_dict, axis=1)
        print(team, 'After', list(transactions_df.columns))

1 Before ['trans_date', 'customer_number', 'SKU', 'price', 'left_in_stock', 'cases_bought']
1 After ['date', 'customer_number', 'sku', 'sale_price', 'items_left', 'total_cases_ordered']
2 Before ['trans_date', 'customer_ID', 'SKU', 'salePrice', 'itemsLeft', 'caseOrdered']
2 After ['date', 'customer_number', 'sku', 'sale_price', 'items_left', 'total_cases_ordered']
3 Before ['Date', 'Customer #', 'SKU', 'Sale Price', 'Items Left', 'Total Cases Ordered']
3 After ['date', 'customer_number', 'sku', 'sale_price', 'items_left', 'total_cases_ordered']


In [ ]:
transactions_columns_and_type_dict = {
    'store': 'INTEGER',
    'date': 'TEXT',
    'customer_number': 'INTEGER',
    'sku': 'INTEGER',
    'sale_price': 'REAL',
    'items_left': 'INTEGER',
    'total_cases_ordered': 'INTEGER'
}

with SQLTableMaker('./databases/region_a.db', 'transactions', transactions_columns_and_type_dict) as sql_table_maker:
    sql_table_maker.add_table()
    
    for team, (table, column_name_dict) in table_and_column_names_dict.items():
    
        database_address = f'./../hw5/databases/Team{team}_grocery.db'
    
        with SQLQueryExecuter(database_address) as sql_query_executer:

            query = f'''
            SELECT *
            FROM {table}
            '''
    
            transactions_df = sql_query_executer.get_query_results(query)
            transactions_df = transactions_df.rename(column_name_dict, axis=1)
            transactions_df['store'] = team
    
        column_names = ', '.join([f'"{column}"' for column in transactions_df.columns])

        list_of_tuples = transactions_df.itertuples(index=False, name=None)
        sql_table_maker.add_multiple_rows_to_table_from_column_names_and_list_of_value_tuples(column_names, len(transactions_df.columns), list_of_tuples)

        sql_table_maker.commit() 

In [11]:
region_a_database_address = './databases/region_a.db'

In [12]:
with SQLQueryExecuter(region_a_database_address) as sql_query_executer:
    
    query = '''
    SELECT name
    FROM sqlite_master 
    WHERE type ='table' 
        AND name NOT LIKE 'sqlite_%';
    '''
    
    tables_df = sql_query_executer.get_query_results(query)
    
tables_df

,name
0,transactions


In [14]:
with SQLQueryExecuter(region_a_database_address) as sql_query_executer:

    query = '''
    SELECT *
    FROM transactions
    '''
    
    transactions_df = sql_query_executer.get_query_results(query)

transactions_df

,store,date,customer_number,sku,sale_price,items_left,total_cases_ordered
0,1,2020-01-01,1,42355001,4.06,179,15
1,1,2020-01-01,1,42562001,2.96,179,15
2,1,2020-01-01,1,44009001,1.31,23,2
3,1,2020-01-01,1,44136001,2.92,23,2
4,1,2020-01-01,1,42813001,1.86,23,2
...,...,...,...,...,...,...,...
36067284,3,20201231,378315,44056001,1.93,31,638
36067285,3,20201231,378315,43770001,0.56,36,642
36067286,3,20201231,378315,44073001,1.46,23,630
36067287,3,20201231,378315,43126001,2.51,31,634


In [15]:
transactions_df['store'].value_counts()

3    13415068
2    11328065
1    11324156
Name: store, dtype: int64

### top 25 for validation


In [16]:
times_sold_df = (transactions_df
                     .groupby(['sku', 'store'])['date']
                     .count()
                     .sort_values(ascending=False)
                     .unstack()
                     .reset_index()
                     .rename({'sku': 'SKU'}, axis=1)
                     .merge(products_df[['Product Name', 'Size', 'itemType', 'SKU']], on=['SKU'])

                )

times_sold_df.head(25)

,SKU,1,2,3,Product Name,Size,itemType
0,42081001,6245,6356,7575,Jambalaya Rice Mix,12 oz,Rice/Rice Mix
1,42082001,6433,6276,7441,Jambalaya Rice Mix,8 oz,Rice/Rice Mix
2,42083001,6419,6279,7598,Guacamole Regular,8 oz,NaN
3,42084001,6285,6234,7626,Coffee Original Blend,12 oz,Coffee/Creamer
4,42085001,6378,6305,7587,GoGurt Variety Pack,8 ct,Yogurt
5,42086001,6249,6357,7510,Italian Dressing,16 oz,Salad Dressing
6,42087001,6385,6209,7622,Cheeseburger Heat & Serve Sliders,29.28 oz,NaN
7,42088001,6281,6212,7530,Choice Cuts Poultry,36 oz,Pet Food
8,42089001,6350,6281,7646,Farmers Pick Concord Grape,46 oz,NaN
9,42090001,6411,6387,7528,Juice Red Grape,64 oz,Juice


In [17]:
top_items_sku_list = []

for store in range(1, 4):
    top_items_sku_list.extend(list(times_sold_df.sort_values(store).tail(25)['SKU'].values))

top_25_from_each_store_df = times_sold_df[times_sold_df['SKU'].isin(top_items_sku_list)].copy()
top_25_from_each_store_df['Total Sold'] = top_25_from_each_store_df[[1, 2, 3]].sum(axis=1)
top_25_from_each_store_df.sort_values('Total Sold', ascending=False)

,SKU,1,2,3,Product Name,Size,itemType,Total Sold
279,42360001,43345,43649,44207,Whole Milk Milk,1/2 gal,Milk,131201
276,42357001,43549,43235,44334,2.00% Milk,1 gal,Milk,131118
275,42356001,43660,43437,43920,1.00% Milk,1/2 gal,Milk,131017
278,42359001,43143,43509,44210,Whole Milk Milk,1 gal,Milk,130862
277,42358001,43368,43077,44027,2.00% Milk,1/2 gal,Milk,130472
274,42355001,43135,43163,44113,1.00% Milk,1 gal,Milk,130411
233,42314001,12193,12449,12906,Squeeze Jelly Grape,20 oz,Jelly/Jam,37548
232,42313001,12257,12413,12797,Jelly Grape,18 oz,Jelly/Jam,37467
231,42312001,12315,12568,12496,Jam Strawberry,18 oz,Jelly/Jam,37379
230,42311001,12155,12407,12631,Jam Grape,18 oz,Jelly/Jam,37193
